In [449]:
import pymysql
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from glob import glob

## Q at V

In [458]:
datafilespath = glob('EIS/raw_data/form/*.mpr') # change to raw data dir
datafilespath =  [os.path.basename(x) for x in datafilespath]
database = np.unique([x.split('_')[0] for x in datafilespath])

In [457]:
# query data from MySQL
##cell001, gcpl, structure 2
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='Acakeren13!'
connection_name='SSB_Modelling'
db=database



V_Interval = [1, 1.5, 2]

for cell in db:
    Connection = pymysql.connect(host=dbhost,
                          port=dbport,
                          user=dbuser,
                          password =dbpassword,
                          database=cell)

    dfs = []
    for i in V_Interval:     
        #formation
        sql = 'SELECT `Ewe/V`, `cycle_Nr`, `Q charge/discharge/mA.h`, `half cycle` FROM {}.form_gcpl'.format(cell)
        cell_df = pd.read_sql(sql, Connection)
        cell_Qdischarge = cell_df[cell_df['half cycle'] == 1]
        cell_Qcharge = cell_df[cell_df['half cycle'] == 0]
        cycle_Nr = np.unique(cell_df['cycle_Nr'])

        QatV_form = pd.DataFrame(index=cycle_Nr)
        charge = []
        discharge = []
        charge_col = 'Q at {}V charge'.format(i)
        discharge_col = 'Q at {}V discharge'.format(i)
        for cycle in cycle_Nr:
            charge.append(cell_Qcharge['Q charge/discharge/mA.h'][(cell_Qcharge['Ewe/V'] > i) & (cell_Qcharge['cycle_Nr'] == cycle)].min())
            discharge.append(cell_Qdischarge['Q charge/discharge/mA.h'][(cell_Qdischarge['Ewe/V'] > i) & (cell_Qdischarge['cycle_Nr'] == cycle)].min())

        QatV_form[charge_col] = charge
        QatV_form[discharge_col] = discharge


        #cycling
        sql = sql = 'SELECT `Ewe/V`, `cycle_Nr`, `Q charge/discharge/mA.h`, `half cycle` FROM {}.cyc_gcpl'.format(cell)
        cell_df = pd.read_sql(sql, Connection)
        cell_Qdischarge = cell_df[cell_df['half cycle'] == 1]
        cell_Qcharge = cell_df[cell_df['half cycle'] == 0]
        cycle_Nr = np.unique(cell_df['cycle_Nr'])
        QatV_cyc = pd.DataFrame(index=cycle_Nr)
        charge = []
        discharge = []
        charge_col = 'Q at {}V charge'.format(i)
        discharge_col = 'Q at {}V discharge'.format(i)
        for cycle in cycle_Nr:
            charge.append(cell_Qcharge['Q charge/discharge/mA.h'][(cell_Qcharge['Ewe/V'] > i) & (cell_Qcharge['cycle_Nr'] == cycle)].min())
            discharge.append(cell_Qdischarge['Q charge/discharge/mA.h'][(cell_Qdischarge['Ewe/V'] > i) & (cell_Qdischarge['cycle_Nr'] == cycle)].min())

        QatV_cyc[charge_col] = charge
        QatV_cyc[discharge_col] = discharge

        dfs.append(pd.concat([QatV_form, QatV_cyc]))

    QatV = pd.concat([dfs[0], dfs[1], dfs[2]], axis=1)
    QatV.index.name = 'cycle'
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=cell, user=dbuser, pw=dbpassword))
    QatV.to_sql('q_at_v', engine, if_exists='replace')
    print('created table q_at_v for {}'.format(cell))
print('success!')

created table q_at_v for Cell001
created table q_at_v for Cell002
created table q_at_v for Cell003
created table q_at_v for Cell004
created table q_at_v for Cell005
created table q_at_v for Cell006
created table q_at_v for Cell007
created table q_at_v for Cell008
success!


In [ ]:
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='Acakeren13!'
connection_name='SSB_Modelling'
db=np.unique([x.split('_')[0] for x in datafilespath])

for cell in db:
    Connection = pymysql.connect(host=dbhost,
                                  port=dbport,
                                  user=dbuser,
                                  password=dbpassword,
                                  database=cell)
    #formation
    sql = 'SELECT `(Q-Qo)/mA.h`,`cycle_Nr`, `half cycle` FROM ' +cell+ '.form_gcpl'
    cellx_df = pd.read_sql(sql, Connection)
    cycle_Nr = np.unique(cellx_df['cycle_Nr'])
    charge=[]
    discharge=[]
    for cycle in cycle_Nr:
        charge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==0)]['(Q-Qo)/mA.h'].max())
        discharge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==1)]['(Q-Qo)/mA.h'].min())
    capacity_form = {'cycle_Nr':cycle_Nr,'Q_charge':charge,'Q_discharge':discharge,'CE/%':np.round(np.divide(np.abs(discharge),charge)*100,2)}
    capacity_form['cycle type'] = 'Formation'
    capacity_form = pd.DataFrame(data=capacity_form)


    
    #cycling
    sql = 'SELECT `Q charge/discharge/mA.h`,`cycle_Nr`, `half cycle` FROM ' +cell+ '.cyc_gcpl'
    cellx_df = pd.read_sql(sql, Connection)
    cycle_Nr = np.unique(cellx_df['cycle_Nr'])
    charge=[]
    discharge=[]
    for cycle in cycle_Nr:
        charge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==0)]['Q charge/discharge/mA.h'].max())
        discharge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==1)]['Q charge/discharge/mA.h'].min())
    capacity_cycle = {'cycle_Nr':cycle_Nr,'Q_charge':charge,'Q_discharge':discharge,'CE/%':np.round(np.divide(np.abs(discharge),charge)*100,2)}
    capacity_cycle['cycle type'] = 'Cycling'
    capacity_cycle = pd.DataFrame(data=capacity_cycle,index=cycle_Nr)
    engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=cell, user=dbuser, pw=dbpassword))
    
    capacity = pd.concat([capacity_form,capacity_cycle])
    capacity.to_sql('capacity', engine, if_exists='replace',index=False)
    print('table capacity of '+cell +' created')

In [ ]:
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='Acakeren13!'
connection_name='SSB_Modelling'
cell='cell001'


Connection = pymysql.connect(host=dbhost,
                              port=dbport,
                              user=dbuser,
                              password=dbpassword,
                              database=cell)
#formation
sql = 'SELECT `(Q-Qo)/mA.h`,`cycle_Nr`, `half cycle` FROM form_gcpl'
cellx_df = pd.read_sql(sql, Connection)
cycle_Nr = np.unique(cellx_df['cycle_Nr'])
charge=[]
discharge=[]
for cycle in cycle_Nr:
    charge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==0)]['(Q-Qo)/mA.h'].max())
    discharge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==1)]['(Q-Qo)/mA.h'].min())
capacity_form = {'cycle_Nr':cycle_Nr,'Q_charge':charge, 'delta_Q_charge'
                 'Q_discharge':discharge,'CE/%':np.round(np.divide(np.abs(discharge),charge)*100,2)}
capacity_form['cycle type'] = 'Formation'
capacity_form = pd.DataFrame(data=capacity_form)



#cycling
sql = 'SELECT `Q charge/discharge/mA.h`,`cycle_Nr`, `half cycle` FROM cyc_gcpl'
cellx_df = pd.read_sql(sql, Connection)
cycle_Nr = np.unique(cellx_df['cycle_Nr'])
charge=[]
discharge=[]
for cycle in cycle_Nr:
    charge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==0)]['Q charge/discharge/mA.h'].max())
    discharge.append(cellx_df[(cellx_df['cycle_Nr']==cycle) & (cellx_df['half cycle']==1)]['Q charge/discharge/mA.h'].min())
capacity_cycle = {'cycle_Nr':cycle_Nr,'Q_charge':charge,'Q_discharge':discharge,'CE/%':np.round(np.divide(np.abs(discharge),charge)*100,2)}
capacity_cycle['cycle type'] = 'Cycling'
capacity_cycle = pd.DataFrame(data=capacity_cycle,index=cycle_Nr)
engine =create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=dbhost, db=cell, user=dbuser, pw=dbpassword))

capacity = pd.concat([capacity_form,capacity_cycle])
capacity.to_sql('capacity_test', engine, if_exists='replace',index=False)
print('table capacity of '+cell +' created')

In [503]:
dbhost='127.0.0.1'
dbport=3306
dbuser='root'
dbpassword='Acakeren13!'
connection_name='SSB_Modelling'
cell='cell001'


Connection = pymysql.connect(host=dbhost,
                              port=dbport,
                              user=dbuser,
                              password=dbpassword,
                              database=cell)
sql = 'SELECT * FROM capacity'
capacity_df = pd.read_sql(sql, Connection)

In [504]:
capacity_df['ΔQ_charge'] = capacity_df['Q_charge'].diff().fillna(0)
capacity_df['ΔQ_discharge'] = capacity_df['Q_discharge'].diff().fillna(0)
capacity_df['ΔQ_charge/%'] = np.round(((capacity_df['Q_charge'].diff().fillna(0)) / capacity_df['Q_charge'])* 100, 2)
capacity_df['ΔQ_discharge/%'] = np.round(((capacity_df['Q_discharge'].diff().fillna(0)) / capacity_df['Q_discharge'])* 100, 2)
capacity_df['ΔCE/%'] = capacity_df['CE/%'].diff().fillna(0)

In [505]:
capacity_df

,cycle_Nr,Q_charge,Q_discharge,CE/%,cycle type,ΔQ_charge,ΔQ_discharge,ΔQ_charge/%,ΔQ_discharge/%,ΔCE/%
0,1,0.629991,-0.315948,50.15,Formation,0.000000,0.000000,0.00,-0.00,0.00
1,2,0.672673,-0.349618,51.97,Formation,0.042681,-0.033670,6.35,9.63,1.82
2,3,0.528947,-0.357295,67.55,Formation,-0.143726,-0.007677,-27.17,2.15,15.58
3,4,0.414435,-0.320788,77.40,Formation,-0.114512,0.036507,-27.63,-11.38,9.85
4,5,0.352402,-0.291690,82.77,Formation,-0.062033,0.029098,-17.60,-9.98,5.37
5,6,0.321627,-0.273272,84.97,Formation,-0.030775,0.018418,-9.57,-6.74,2.20
6,7,0.299554,-0.257594,85.99,Formation,-0.022074,0.015678,-7.37,-6.09,1.02
7,8,0.280395,-0.244698,87.27,Formation,-0.019159,0.012897,-6.83,-5.27,1.28
8,9,0.262684,-0.232620,88.56,Formation,-0.017711,0.012078,-6.74,-5.19,1.29
9,10,0.246246,-0.221435,89.92,Formation,-0.016438,0.011185,-6.68,-5.05,1.36
